In [1]:
import os
import json
import random
from pathlib import Path
from zipfile import ZipFile

In [2]:
import pandas as pd 
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import string
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
import plotly.graph_objects as go
import plotly.express as px

In [3]:
df = pd.read_csv('../../Resources/communes_commentaire_tmp.csv'); df.head()

,Insee,Code_postal,Ville,Sécurité,Éducation,Sport / Loisir,Environnement,Vie pratique,pouce_positif,pouce_negatif,commentaire,date,auteur
0,25002,25440,Abbans-Dessus,2.0,4.0,2.0,3.0,3.0,0,0,La Ville et les hameaux qui pourrait être agr...,27/02/2022,Hitoluc4
1,44001,44170,Abbaretz,4.0,5.0,3.0,3.0,5.0,1,0,Abbaretz c'est une commune qui est très propre...,22/11/2020,Zera
2,44001,44170,Abbaretz,5.0,5.0,5.0,5.0,5.0,0,0,"Village au top, un joli lac pour se promener e...",01/10/2021,ptitepaolina
3,44001,44170,Abbaretz,5.0,3.0,3.0,5.0,3.0,0,0,Abbaretz est une ville de taille moyenne situé...,18/03/2020,Thomas
4,44001,44170,Abbaretz,4.0,5.0,4.0,4.0,4.0,1,0,Une ville qui fait un maximun pour les citoyens,16/06/2017,Lara


In [4]:
data = df.copy()
data.shape

(11456, 13)

In [5]:
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
3,Sécurité,2.723464
4,Éducation,2.723464
5,Sport / Loisir,2.723464
6,Environnement,2.723464
7,Vie pratique,2.723464
0,Insee,0.000000
1,Code_postal,0.000000
2,Ville,0.000000
8,pouce_positif,0.000000
9,pouce_negatif,0.000000


In [6]:
data = data.dropna()

In [7]:
data.shape

(11144, 13)

In [8]:
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
0,Insee,0.0
1,Code_postal,0.0
2,Ville,0.0
3,Sécurité,0.0
4,Éducation,0.0
5,Sport / Loisir,0.0
6,Environnement,0.0
7,Vie pratique,0.0
8,pouce_positif,0.0
9,pouce_negatif,0.0


In [9]:
data["commentaire"]

0        La Ville et les  hameaux qui pourrait être agr...
1        Abbaretz c'est une commune qui est très propre...
2        Village au top, un joli lac pour se promener e...
3        Abbaretz est une ville de taille moyenne situé...
4          Une ville qui fait un maximun pour les citoyens
                               ...                        
11451    Cadre de vie assez agréable mais malheureuseme...
11452    Ville ni trop grande ni trop petite permettant...
11453    Pas assez de commerces, pas assez d'animation,...
11454    J'aime bien ma ville qui comprend beaucoup d'e...
11455    Ville calme, propre, entouré de campagne et tr...
Name: commentaire, Length: 11144, dtype: object

In [10]:
data.iloc[:,10]

0        La Ville et les  hameaux qui pourrait être agr...
1        Abbaretz c'est une commune qui est très propre...
2        Village au top, un joli lac pour se promener e...
3        Abbaretz est une ville de taille moyenne situé...
4          Une ville qui fait un maximun pour les citoyens
                               ...                        
11451    Cadre de vie assez agréable mais malheureuseme...
11452    Ville ni trop grande ni trop petite permettant...
11453    Pas assez de commerces, pas assez d'animation,...
11454    J'aime bien ma ville qui comprend beaucoup d'e...
11455    Ville calme, propre, entouré de campagne et tr...
Name: commentaire, Length: 11144, dtype: object

In [11]:
data['Comment'] = data['commentaire'].apply(lambda f: f.lower())

In [12]:
AComment=[]
for comment in data["Comment"].apply(str):
    Word_Tok = []
    for word in  re.sub("\W"," ",comment ).split():
        Word_Tok.append(word)
    AComment.append(Word_Tok)

In [13]:
data["Word_Tok"] = AComment
data.head()

,Insee,Code_postal,Ville,Sécurité,Éducation,Sport / Loisir,Environnement,Vie pratique,pouce_positif,pouce_negatif,commentaire,date,auteur,Comment,Word_Tok
0,25002,25440,Abbans-Dessus,2.0,4.0,2.0,3.0,3.0,0,0,La Ville et les hameaux qui pourrait être agr...,27/02/2022,Hitoluc4,la ville et les hameaux qui pourrait être agr...,"[la, ville, et, les, hameaux, qui, pourrait, ê..."
1,44001,44170,Abbaretz,4.0,5.0,3.0,3.0,5.0,1,0,Abbaretz c'est une commune qui est très propre...,22/11/2020,Zera,abbaretz c'est une commune qui est très propre...,"[abbaretz, c, est, une, commune, qui, est, trè..."
2,44001,44170,Abbaretz,5.0,5.0,5.0,5.0,5.0,0,0,"Village au top, un joli lac pour se promener e...",01/10/2021,ptitepaolina,"village au top, un joli lac pour se promener e...","[village, au, top, un, joli, lac, pour, se, pr..."
3,44001,44170,Abbaretz,5.0,3.0,3.0,5.0,3.0,0,0,Abbaretz est une ville de taille moyenne situé...,18/03/2020,Thomas,abbaretz est une ville de taille moyenne situé...,"[abbaretz, est, une, ville, de, taille, moyenn..."
4,44001,44170,Abbaretz,4.0,5.0,4.0,4.0,4.0,1,0,Une ville qui fait un maximun pour les citoyens,16/06/2017,Lara,une ville qui fait un maximun pour les citoyens,"[une, ville, qui, fait, un, maximun, pour, les..."


In [14]:
stop_words=set(STOP_WORDS)

deselect_stop_words = ['n\'', 'ne','pas','plus','personne','aucun','ni','aucune','rien']
for w in deselect_stop_words:
    if w in stop_words:
        stop_words.remove(w)
    else:
        continue

In [15]:
AllfilteredComment=[]
for comment in data["Word_Tok"]:
    filteredComment = [w for w in comment if not ((w in stop_words) or (len(w) == 1))]
    AllfilteredComment.append(' '.join(filteredComment))

In [16]:
data["CommentAferPreproc"]=AllfilteredComment
data.head()

,Insee,Code_postal,Ville,Sécurité,Éducation,Sport / Loisir,Environnement,Vie pratique,pouce_positif,pouce_negatif,commentaire,date,auteur,Comment,Word_Tok,CommentAferPreproc
0,25002,25440,Abbans-Dessus,2.0,4.0,2.0,3.0,3.0,0,0,La Ville et les hameaux qui pourrait être agr...,27/02/2022,Hitoluc4,la ville et les hameaux qui pourrait être agr...,"[la, ville, et, les, hameaux, qui, pourrait, ê...",ville hameaux agréable vivre 5ans mairie laiss...
1,44001,44170,Abbaretz,4.0,5.0,3.0,3.0,5.0,1,0,Abbaretz c'est une commune qui est très propre...,22/11/2020,Zera,abbaretz c'est une commune qui est très propre...,"[abbaretz, c, est, une, commune, qui, est, trè...",abbaretz commune propre déchets vie pratique t...
2,44001,44170,Abbaretz,5.0,5.0,5.0,5.0,5.0,0,0,"Village au top, un joli lac pour se promener e...",01/10/2021,ptitepaolina,"village au top, un joli lac pour se promener e...","[village, au, top, un, joli, lac, pour, se, pr...",village top joli lac promener commerces essent...
3,44001,44170,Abbaretz,5.0,3.0,3.0,5.0,3.0,0,0,Abbaretz est une ville de taille moyenne situé...,18/03/2020,Thomas,abbaretz est une ville de taille moyenne situé...,"[abbaretz, est, une, ville, de, taille, moyenn...",abbaretz ville taille moyenne située pays chât...
4,44001,44170,Abbaretz,4.0,5.0,4.0,4.0,4.0,1,0,Une ville qui fait un maximun pour les citoyens,16/06/2017,Lara,une ville qui fait un maximun pour les citoyens,"[une, ville, qui, fait, un, maximun, pour, les...",ville maximun citoyens


In [17]:
senti_list = []
for i in data["CommentAferPreproc"]:
    vs = tb(i).sentiment[0]
    if (vs > 0):
        senti_list.append('Positive')
    elif (vs < 0):
        senti_list.append('Negative')
    else:
        senti_list.append('Neutral')

> tb = textBlob.Blobber()

In [18]:
data["sentiment"] = senti_list
data.head()

,Insee,Code_postal,Ville,Sécurité,Éducation,Sport / Loisir,Environnement,Vie pratique,pouce_positif,pouce_negatif,commentaire,date,auteur,Comment,Word_Tok,CommentAferPreproc,sentiment
0,25002,25440,Abbans-Dessus,2.0,4.0,2.0,3.0,3.0,0,0,La Ville et les hameaux qui pourrait être agr...,27/02/2022,Hitoluc4,la ville et les hameaux qui pourrait être agr...,"[la, ville, et, les, hameaux, qui, pourrait, ê...",ville hameaux agréable vivre 5ans mairie laiss...,Positive
1,44001,44170,Abbaretz,4.0,5.0,3.0,3.0,5.0,1,0,Abbaretz c'est une commune qui est très propre...,22/11/2020,Zera,abbaretz c'est une commune qui est très propre...,"[abbaretz, c, est, une, commune, qui, est, trè...",abbaretz commune propre déchets vie pratique t...,Positive
2,44001,44170,Abbaretz,5.0,5.0,5.0,5.0,5.0,0,0,"Village au top, un joli lac pour se promener e...",01/10/2021,ptitepaolina,"village au top, un joli lac pour se promener e...","[village, au, top, un, joli, lac, pour, se, pr...",village top joli lac promener commerces essent...,Positive
3,44001,44170,Abbaretz,5.0,3.0,3.0,5.0,3.0,0,0,Abbaretz est une ville de taille moyenne situé...,18/03/2020,Thomas,abbaretz est une ville de taille moyenne situé...,"[abbaretz, est, une, ville, de, taille, moyenn...",abbaretz ville taille moyenne située pays chât...,Neutral
4,44001,44170,Abbaretz,4.0,5.0,4.0,4.0,4.0,1,0,Une ville qui fait un maximun pour les citoyens,16/06/2017,Lara,une ville qui fait un maximun pour les citoyens,"[une, ville, qui, fait, un, maximun, pour, les...",ville maximun citoyens,Neutral
